In [3]:
import numpy as np
import cv2
import glob
import os

os.makedirs("calibration_results", exist_ok=True)

pattern_size = (4, 7)  
square_size = 2.5   

objp = np.zeros((pattern_size[0] * pattern_size[1], 3), np.float32)
objp[:, :2] = np.mgrid[0:pattern_size[0], 0:pattern_size[1]].T.reshape(-1, 2)
objp *= square_size  

obj_points = [] 
img_points = []  

calib_images = glob.glob("calib_images/calib_*.JPG")
assert len(calib_images) > 0, "未找到标定图像，请检查calib_images目录"

for fname in calib_images:
    img = cv2.imread(fname)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    ret, corners = cv2.findChessboardCorners(gray, pattern_size)
    
    if ret:
        corners_refined = cv2.cornerSubPix(
            gray, corners, (11, 11), (-1, -1),
            criteria=(cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.001)
        )
        
        obj_points.append(objp)
        img_points.append(corners_refined)
        
        cv2.drawChessboardCorners(img, pattern_size, corners_refined, ret)
        cv2.imshow("Corners", img)
        cv2.waitKey(500)  # 显示0.5秒

cv2.destroyAllWindows()

assert len(obj_points) >= 10, f"有效标定图不足（当前{len(obj_points)}张），至少需要10张"

ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(
    obj_points, img_points, gray.shape[::-1], None, None
)

np.savez("calibration_results/calibration_data.npz", 
         mtx=mtx, dist=dist, rvecs=rvecs, tvecs=tvecs)

mean_error = 0
for i in range(len(obj_points)):
    img_points2, _ = cv2.projectPoints(obj_points[i], rvecs[i], tvecs[i], mtx, dist)
    error = cv2.norm(img_points[i], img_points2, cv2.NORM_L2) / len(img_points2)
    mean_error += error
print(f"平均重投影误差: {mean_error/len(obj_points):.3f} 像素（应<0.5像素）")


平均重投影误差: 0.306 像素（应<0.5像素）


In [5]:
test_img = cv2.imread(calib_images[0])
h, w = test_img.shape[:2]
new_mtx, roi = cv2.getOptimalNewCameraMatrix(mtx, dist, (w, h), 1, (w, h))
undistorted = cv2.undistort(test_img, mtx, dist, None, new_mtx)

cv2.imwrite("calibration_results/original_vs_undistorted.jpg", 
           np.hstack((test_img, undistorted)))
print("校准完成！结果已保存到 calibration_results/ 目录")

校准完成！结果已保存到 calibration_results/ 目录


In [ ]:
#检测有效标定图
import cv2
img = cv2.imread("calib_images/calib_01.JPG")
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
pattern_size = (4, 7)  # 8x6方格的棋盘格内角点是7x5
ret, corners = cv2.findChessboardCorners(gray, pattern_size)

if ret:
    # 可视化角点
    cv2.drawChessboardCorners(img, pattern_size, corners, ret)
    cv2.imshow("Corners", img)
    cv2.waitKey(0)
else:
    print("角点检测失败！检查：")
    print("1. pattern_size是否正确")
    print("2. 棋盘格是否完整可见")